In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import proplot as pplt
import seaborn as sns
from tqdm import tqdm, trange

sys.path.append('/Users/46h/Research/') 
from scdist.tools import ap_utils
from scdist.tools import utils

In [ ]:
pplt.rc['axes.grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

## Setup 

Define reconstruction grid.

In [ ]:
bins = 50
limits = [(-6.0, 6.0), (-6.0, 6.0)]
binwidths = np.diff(limits)[:, 0] / bins
print(binwidths)

Create the true distribution.

In [ ]:
X_true = np.random.multivariate_normal(mean=[0., 0.], cov=[[1.0, 0.4], [0.4, 1.0]], size=100000)

In [ ]:
fig, ax = pplt.subplots()
sns.histplot(ax=ax, x=X_true[:, 0], y=X_true[:, 1], bins=bins, binrange=limits)
plt.show()

Take measurements.

In [ ]:
xmax_screen = 10.0
bins_screen = 50
screen_edges = np.linspace(-xmax_screen, xmax_screen, bins_screen)

In [ ]:
np.random.seed(3)
n_proj = 10
angles = np.linspace(0, np.pi, n_proj, endpoint=False)
betas = np.random.uniform(0.5, 2.0, size=(n_proj,))
alphas = np.random.uniform(-0.5, 0.5, size=(n_proj,))

tmats = []
for i in range(n_proj):
    R = utils.rotation_matrix(angles[i])
    V = ap_utils.V_matrix_2x2(alphas[i], betas[i])
    M = np.matmul(V, R)
    tmats.append(M)
    
projections = []
for M in tqdm(tmats):
    X_meas = utils.apply(M, X_true)
    sns.jointplot(x=X_meas[:, 0], y=X_meas[:, 1], kind='hist', height=2.5, bins=bins_screen,
                  xlim=(-xmax_screen, xmax_screen), ylim=(-5.0, 5.0))
    plt.show()
    projection, _ = np.histogram(X_meas[:, 0], screen_edges)
    projection = projection / np.sum(projection)
    projections.append(projection)

In [ ]:
projections_true = np.copy(projections)

In [ ]:
fig, ax = pplt.subplots()
ax.pcolormesh(projections_true, ec='None', colorbar=True)
plt.show()

## Reconstruction 

Bin true distribution for comparison.

In [ ]:
Z_true, _, _ = np.histogram2d(X_true[:, 0], X_true[:, 1], bins, limits)
Z_true /= np.sum(Z_true)

Initialize reconstruction bunch.

In [ ]:
widths = np.diff(limits)[:, 0] / bins
widths *= 1.0
print(widths)

In [ ]:
n_parts = 100000
mins = [lim[0] for lim in limits]
maxs = [lim[1] for lim in limits]
X = np.random.uniform(mins, maxs, size=(n_parts, 2))

In [ ]:
proj_errors = []
true_errors = []

for iteration in range(6):
    # Simulate the measurements.
    projections, coords = [], []
    for M in tqdm(tmats):
        X_meas = utils.apply(M, X)
        projection, _ = np.histogram(X_meas[:, 0], bins=screen_edges)
        projection = projection / np.sum(projection)
        projections.append(projection)
        coords.append(X_meas)
    projections = np.array(projections)
    coords = np.array(coords)

#     fig, axes = pplt.subplots(ncols=2,)
#     axes[0].pcolormesh(projections_true, ec='None')
#     axes[1].pcolormesh(projections, ec='None')
#     axes[0].set_title('True')
#     axes[1].set_title('Reconstructed')
#     plt.show()

    # Weight particles.
    weights = np.zeros((n_proj, X.shape[0]))
    for k in range(n_proj):
        xidx = np.digitize(coords[k, :, 0], screen_edges) - 1
        on_screen = np.logical_and(xidx >= 0, xidx < len(screen_edges) - 1)
        weights[k, on_screen] = projections_true[k, xidx[on_screen]]
    
    # Only keep particles that hit every screen.
    keep_idx = [np.all(weights[:, i] > 0.) for i in range(weights.shape[1])]
    weights[:, np.logical_not(keep_idx)] = 0.
    weights = np.sum(weights, axis=0)
    
    
    for i, weight in enumerate(weights):
        
    
    
    weights /= np.sum(weights)
    
    # Convert the weights to counts.
    counts = weights * n_parts
    counts = np.round(counts).astype(int)

#     fig, ax = pplt.subplots()
#     ax.scatter(X[:, 0], X[:, 1], s=1, c=counts, 
#                colorbar=True, colorbar_kw=dict(label='Number of new particles to be added'))
#     plt.show()

    # Generate a new bunch.
    add_idx = counts > 0
    lo = np.repeat(X[add_idx] - 0.5 * widths, counts[add_idx], axis=0)
    hi = np.repeat(X[add_idx] + 0.5 * widths, counts[add_idx], axis=0)
    X = np.random.uniform(lo, hi)

    fig, axes = pplt.subplots(ncols=2)
    for ax, _X in zip(axes, [X_true, X]):
        sns.histplot(x=_X[:, 0], y=_X[:, 1], ax=ax, ec='None', 
                     bins=bins, binrange=limits, colorbar=True, 
                     thresh=None, cmap='viridis',
                    )
    axes.format(xlabel="x", ylabel="x'")
    axes[0].set_title('True')
    axes[1].set_title('Reconstructed')
    plt.show()

    proj_error = np.sum((projections - projections_true)**2)
    proj_errors.append(proj_error)
    
    Z_rec, _, _ = np.histogram2d(X[:, 0], X[:, 1], bins, limits)
    Z_rec /= np.sum(Z_rec)
    true_error = np.sqrt(np.sum((Z_true - Z_rec)**2))
    true_errors.append(true_error)
    
    print('proj_error = {}'.format(proj_error))
    print('true_error = {}'.format(true_error))
    print('New bunch has {} particles'.format(X.shape[0]))
    print('Iteration {} complete'.format(iteration))
    print()

In [ ]:
fig, axes = pplt.subplots(nrows=2, figsize=(3, 3))
kws = dict(marker='.', color='black')
axes[0].plot(true_errors, **kws)
axes[1].plot(proj_errors, **kws)
plt.show()

I believe you need to reduce the weights by the number of particles already in that bin.